# 0. Общий код

Ячейка подключает библиотеки и задаёт функции, используемые в ячейках 1.1 и далее, для сокращения объёма кода.

In [25]:
# 0. Общий код

# noinspection PyUnresolvedReferences
from lib import *


# -------------------------- VARS --------------------------------
# Symbols and combinations that have to be corrected after OCR
COMBINATIONS_CORR_ALPHABET = {
    'A': 'А', 'a': 'а', 'B': 'В', 'b': 'Ь', 'C': 'С', 'c': 'с', 'E': 'Е', 'e': 'е', 'H': 'Н', 'K': 'К', 'M': 'М',
    'O': 'О', 'P': 'Р', 'p': 'р', 'T': 'Т', 'X': 'Х', 'y': 'у', 'x': 'х',
    'U': 'И',
    'u': 'и',
    'r': 'г',
    'N': 'П',
    'n': 'п',
    'm': 'т',
    'Y': 'У',
    # 'S' : 'Я',		# Seems irrelevant
}
COMBINATIONS_CORR_UNICODE = {
    'І': 'I',  # These two "I" are different!
    'ก': 'п',
    '山': 'Ц',
    'כ': 'э',
    'חи': 'пи',
    'प': 'Ч',
    'иั': 'й'
}
COMBINATIONS_CORR_OTHER = {
    ' -': '-',
    '- ': '-',
    '0': 'О',
    '3': 'З',
    '6': 'б',
}
COMBINATIONS_CORR_GLOBAL = dict_merge(COMBINATIONS_CORR_ALPHABET,
                                      dict_merge(COMBINATIONS_CORR_UNICODE, COMBINATIONS_CORR_OTHER))
# Symbols excluded in xml have to be converted back
XML_EXCLUDES = {
    '&quot;': '"',
    '&apos;': "'",
    '&lt;': '<',
    '&gt;': '>',
    '&amp;': '&'
}
PERSONAL_WORD_LIST = "./matphys/PWL.txt"
URI_PREFIX = "http://libmeta.ru/fme/"
# ----------------------------------------------------------------

# 1. Базовый парсер заголовков

Вытаскивает из latex-кода заголовки статей и их расположение в файлах.

Разбивка происходит в полуручном режиме, т.к. нет уверенности в формате заголовков.

В тексте ищутся слова, содержащие в своём составе заглавные буквы на русском и английском языках в отношении, большем или равным заданному (по умолчанию 0.51, при меньших значениях количество вхождений значительно возрастает, например за счёт двухбуквенных предлогов). Предполагается, что таким образом удаётся обнаруживать неправильно машинно распознанный капс. Слова или цепочки слов, состоящие из одного строчного символа включаются в заголовок, если стоят между слов, определённых как часть заголовка. При этом, одиночные заглавные буквы, а также инициалы не воспринимаются как начало заголовка.

## Использование
- При удовлетворительном определении заголовка нажать `Enter` без дополнительного ввода.
- Если предложенное место заголовком не является ввести `"n"`
- При неправильном определении границ заголовка ввести два корректировочных числа для сдвига левой и правой границы.
  - ЗАМЕЧАНИЕ: сдвиг производится попробельно, т.е. двойной пробел будет распознан как слово нулевой длины.
  - ЗАМЕЧАНИЕ: границы отображаемого фрагмента текста будут передвинуты автоматически. Длины левой и правой границ в словах задаются в параметрах.
  - ПРИМЕРЫ:
    - `out: a [B C] d e f` -> `in: 0 2` -> `out: a [B C D E] f`
    - `out: a b c [D E] f` -> `in: 2 -1` -> `out: a [B C D] e f`
- Также возможен посимвольный сдвиг правой границы в случае "сращивания" заголовка статьи и её текста. Ввести одно число, начиная с точки.
  - ПРИМЕРЫ:
    - `out: a[BC]def` -> `in: .2` -> `out: a[BCDE]f`
    - `out: a[BCDE]f` -> `in: .-1` -> `out: a[BCD]ef`

В выводе в терминале переносы строк для удобства заменены на `"$"`

### Прочее
- Для определителя капса доступны исключения, которые никогда не будут рассматриваться, как потенциальные начала заголовков, см. опции. По умолчанию: первые 10 римских цифр, "МэВ" и "ГэВ". Также определитель не реагирует на "СМ.", что часто встречается в ссылках сразу после заголовков.
- Использовать системный терминал для взаимодействия оказывается удобнее, чем использовать jupyter, поэтому можно скопировать ячейку с кодом в файл `scripter.py` и запускать его.
- При положительном определении заголовка файл дополняется немедленно, прервать процесс можно в любой момент, как и продолжить после -- итоговый файл будет дополняться, а не перезаписываться с нуля при новом запуске программы (главное не забыть предварительно удалить из конца файла дубликаты, если вы начинаете с той страницы, на которой закончили в прошлый раз, а не со следующей).
- В случае пропуска парсером заголовка его можно добавить вручную двумя способами:
  1) Сдвинуть границы заголовка назад, как описано в инструкции выше. Подходит, если была пропущена небольшая (обычно ссылочная) статья, примерно 20 слов, плюс-минус. При этом после ввода заголовка поиск продолжится с __его__ конца, поэтому следующий заголовок "вместо" которого был введён пропущенный будет определён заново и пропущен не будет.
  2) Воспользоваться ячейкой 1.1. Для этого в сыром tex-файле страницы нужно отыскать заголовок, скопировать его и __в точности__ вставить в разделе параметров, а также указать номер страницы. Скрипт парсера при этом можно не закрывать, последующая нумерация подстроится автоматически.

In [ ]:
# 1. Базовый парсер заголовков

# noinspection PyUnresolvedReferences
from os import walk
# noinspection PyUnresolvedReferences
import xml.etree.ElementTree as ElementTree
# noinspection PyUnresolvedReferences
from xml.dom import minidom
# noinspection PyUnresolvedReferences
import re
# noinspection PyUnresolvedReferences
import codecs


############################ VARS ################################
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./matphys/"
EXIT_FILE = "FMEv2.xml"
# First and last pages to be parsed
START_PAGE = 639
END_PAGE = 700
# How many words to display before and after a potential title
LEAD_WORDS = 5
AFT_WORDS = 5
# Look in the description
CAPS_QUOT = 0.51
EXCEPTIONS = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'МэВ', 'ГэВ']
##################################################################



class Article:
    start_title = 0
    end_title = 0
    filename = ''



# Write xml tree to file
def prettify_1(elem_local:ElementTree.Element) -> str:
    # Pretty-printed XML string for the Element.
    rough_string = ElementTree.tostring(elem_local, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")
def xml_write_1(root_local:ElementTree.Element):
    with codecs.open(EXIT_DIR + EXIT_FILE, 'w', 'utf-8') as f_out:
        f_out.write(prettify_1(root_local))


# Get filenames needed
filenames_raw = next(walk(PAGES_DIR), (None, None, []))[2]  # [] if no file
filenames = []
for i in range(START_PAGE, END_PAGE + 1):
    for filename in filenames_raw:
        beginning = "rp-" + str(i) + "_"
        if filename[:len(beginning)] == beginning and filename[-4:] == ".mmd":
            filenames.append(filename)


# Check for existing xml
filenames_raw = next(walk(EXIT_DIR), (None, None, []))[2]  # [] if no file
if not(EXIT_FILE in filenames_raw):
    root = ElementTree.Element('data')
    xml_write_1(root)


# Convert xml excluded symbols
def xml_excluded_convert (text_local:str) -> str:
    for key_local in XML_EXCLUDES.keys():
        while text_local.find(key_local) != -1:
            pos_local = text_local.find(key_local)
            text_local = text_local[:pos_local] + XML_EXCLUDES[key_local] + text_local[pos_local + len(key_local):]
    return text_local
def remove_xml_spaces_1(elem_local:ElementTree.Element) -> ElementTree.Element:
    elem_local.tail = None
    if elem_local.text is not None:
        is_space = True
        for letter in elem_local.text:
            is_space = False if letter != ' ' else is_space
        elem_local.text = None if is_space else xml_excluded_convert(elem_local.text)
    for subelem in elem_local:
        subelem = remove_xml_spaces_1(subelem)
        subelem.tail = None
    return elem_local
def parse_xml_1() -> ElementTree.Element:
    # Parse existing xml (string parsing is needed to avoid extra newlines appearing)
    exit_string = ''
    with codecs.open(EXIT_DIR + EXIT_FILE, 'r', 'utf-8') as f_in:
        for p in f_in.readlines():
            exit_string += p[:-1]
    root_local = ElementTree.fromstring(exit_string)
    # Remove empty tails and texts
    root_local = remove_xml_spaces_1(root_local)
    return root_local
root = parse_xml_1()
num = len(root) + 1


# Add article title and metadata to xml tree
def add_article_1(elem_local:Article) -> int:
    # Update root in case it's been changed
    elem_root = parse_xml_1()
    elem_num = len(elem_root) + 1
    elem_article = ElementTree.SubElement(elem_root, 'article', {'n':str(elem_num)})
    elem_title = ElementTree.SubElement(elem_article, 'title')
    elem_title.text = file[elem_local.start_title + 1:elem_local.end_title]
    elem_title_meta = ElementTree.SubElement(elem_article, 'title-meta')
    elem_title_file = ElementTree.SubElement(elem_title_meta, 'title-file')
    elem_title_file.text = elem_local.filename
    elem_title_start = ElementTree.SubElement(elem_title_meta, 'title-start')
    elem_title_start.text = str(elem_local.start_title + 1)
    elem_title_end = ElementTree.SubElement(elem_title_meta, 'title-end')
    elem_title_end.text = str(elem_local.end_title)
    xml_write_1(elem_root)
    return elem_num


# Count number of alphabetic letters in word
def count_letters_1(word_local:str) -> int:
    number = 0
    for letter in word_local:
        number += 0 if re.match(r"[A-ZА-Яa-zа-я]", letter) is None else 1
    return number

# Check if word is written in CAPS
def check_caps_1(word_local:str) -> int:
    number = 0
    len_word = 0
    while len(word_local) and re.match(r"[!#$%&'*+-.^_`|~:]", word_local[-1]) is not None:
        word_local = word_local[:-1]
    while len(word_local) and re.match(r"[!#$%&'*+-.^_`|~:]", word_local[0]) is not None:
        word_local = word_local[1:]
    for letter in word_local:
        #num += 0 if re.match(r"[A-ZА-Я0-9]|[!#$%&'*+-.^_`|~:]", letter) is None else 1					# Too many symbols, math formulas are being detected
        len_word += 1 if re.match(r"[!#$%&'*+-.^_`|~:]", letter) is None else 0
        number += 0 if re.match(r"[A-ZА-Я]", letter) is None else 1
    return 0 if len_word == 0 or number / len_word < CAPS_QUOT or word_local in EXCEPTIONS else number				# Also exclude common roman numbers

# Check for initials like "I.E."
def check_initials_1(word_to_check:str) -> bool:
    initials = True
    for p in range(len(word_to_check) - 1):
        type_1 = 0 if re.match(r"[A-ZА-Яa-zа-я]", word_to_check[p]) is None else 1
        type_2 = 0 if re.match(r"[A-ZА-Яa-zа-я]", word_to_check[p + 1]) is None else 1
        initials = False if type_1 and type_2 else initials
    return initials

# Check if the word is "CM." which happens often
def check_link_1(word_local:str) -> bool:
    word_local = word_local.upper()
    # Convert to cyrillic
    for p in range(len(word_local)):
        word_local = (word_local[:p] + 'С' + word_local[p + 1:]) if word_local[p] == 'C' else word_local
        word_local = (word_local[:p] + 'М' + word_local[p + 1:]) if word_local[p] == 'M' else word_local
    return True if word_local == 'СМ.' else False


# Find next ot prev word boundary (space / newline)
def prev_from_1(position:int, file_name:str) -> int:
    position = max(position, 0)
    prev_space = file_name.rfind(' ', 0, position)
    prev_nl = file_name.rfind('\n', 0, position)
    prev_space = -1 if prev_space == -1 else prev_space
    prev_nl = -1 if prev_nl == -1 else prev_nl
    return max(prev_nl, prev_space)
def next_from_1(position:int, file_name:str, end_replace = True) -> int:
    next_space = file_name.find(' ', position + 1)
    next_nl = file_name.find('\n', position + 1)
    if end_replace:
        next_space = len(file_name) if next_space == -1 else next_space
        next_nl = len(file_name) if next_nl == -1 else next_nl
    return max(next_nl, next_space) if next_space == -1 or next_nl == -1 else min(next_nl, next_space)


# Main loop
for filename in filenames:
    print()
    print("################################ " + filename + " ################################")
    with codecs.open(PAGES_DIR + filename, 'r', 'utf-8') as f:
        file = f.read()

    word_bound_l = -1
    word_bound_r = next_from_1(word_bound_l, file, end_replace=False)
    EOF_reached = False

    while not EOF_reached:
        if word_bound_r == -1:
            word_bound_r = len(file)
            EOF_reached = True


        if check_caps_1(file[word_bound_l+1:word_bound_r]) < 2 or check_initials_1(file[word_bound_l+1:word_bound_r]) or check_link_1(file[word_bound_l+1:word_bound_r]):
            word_bound_l = word_bound_r
            word_bound_r = next_from_1(word_bound_l, file, end_replace=False)

        else: # Possibly found a title
            # Left border of a title is already known
            start_title = word_bound_l

            # Define right border of a title
            defined_end = False
            end_title = word_bound_r
            while not defined_end:
                word_bound_l = word_bound_r
                word_bound_r = next_from_1(word_bound_l, file)

                if word_bound_l == len(file):
                    defined_end = True
                elif check_link_1(file[word_bound_l+1:word_bound_r]):
                    # A "CM." link, not a title
                    pass
                elif not check_caps_1(file[word_bound_l+1:word_bound_r]) and count_letters_1(file[word_bound_l+1:word_bound_r]) < 2:
                    if re.match(r"[A-ZА-Яa-zа-я]", file[word_bound_l+1]) is not None:
                        # Most possibly belongs to title
                        end_title = word_bound_r
                    else:
                        # Most possibly NOT belongs to title
                        pass
                elif check_caps_1(file[word_bound_l+1:word_bound_r]):
                    end_title = word_bound_r
                else:
                    defined_end = True

            next_title = False
            while not next_title:
                # Update root in case it's been changed
                root = parse_xml_1()
                num = len(root) + 1

                # Console output for further user actions
                segment_start = start_title
                segment_end = end_title
                for i in range(LEAD_WORDS):
                    segment_start = prev_from_1(segment_start, file)
                for i in range(AFT_WORDS):
                    segment_end = next_from_1(segment_end, file)

                out_str = file[segment_start+1:segment_end]

                # Format
                for i in range(len(out_str)):
                    out_str = out_str[:i] + ('$' if out_str[i] == '\n' else out_str[i]) + out_str[i+1:]
                out_str = f"{num})\n" + out_str + '\n' + ' ' * (start_title - segment_start) + '^' * (end_title - start_title - 1)
                # Check for "section" in the string. This is referred to alphabetic tip at the bottom of the page
                """if 'section' in out_str or 'title' in out_str:
                    out_str += '     ############################### Title or section found! ###############################'""" # Not Used
                print(out_str)

                # User actions
                response = input()
                # noinspection PyBroadException
                try:
                    if response == '':
                        # Add article
                        article = Article()
                        article.start_title = start_title
                        article.end_title = end_title
                        article.filename = filename
                        num = add_article_1(article)
                        next_title = True
                        word_bound_l = end_title
                        word_bound_r = next_from_1(word_bound_l, file, end_replace=False)
                        print(f'Adding article, n="{num}", title="{file[start_title+1:end_title]}"\n\n')
                    elif response == 'n' or response == 'т':
                        # Do not add this one
                        next_title = True
                        print("Not an article, skipping\n\n")
                    elif response[0] == '.':
                        end_title += int(response[1:])
                        print("Changing title right border\n\n")
                    else:
                        # Change title borders
                        corrections = response.split(' ')
                        corrections[0] = int(corrections[0])
                        corrections[1] = int(corrections[1])
                        if corrections[0] > 0:
                            for i in range(abs(corrections[0])):
                                start_title = prev_from_1(start_title, file)
                        if corrections[0] < 0:
                            for i in range(abs(corrections[0])):
                                start_title = next_from_1(start_title, file)
                        if corrections[1] < 0:
                            for i in range(abs(corrections[1])):
                                end_title = prev_from_1(end_title, file)
                        if corrections[1] > 0:
                            for i in range(abs(corrections[1])):
                                end_title = next_from_1(end_title, file)
                        print("Changing title borders\n\n")
                except:
                    print("########## !!! Failed on input, try again !!! ##########\n\n")


# End reached
print('###########################################################################################')
print('Last requested page processed. Press "Enter" to close this window.')
response = input()

## 1.1. Добавление заголовков по одному

В разделе параметров указать номер страницы и ТОЧНУЮ формулировку заголовка из сырого latex-текста, а также номер страницы, после чего запустить ячейку.

Закрывать скрипт парсера не обязательно, это не вызовет ошибок и его нумерация подстроится автоматически.

In [ ]:
# 1.1. Добавление заголовков по одному

############################ VARS ################################
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./results/FMEtitles/"
EXIT_FILE = "FMEtitles-added-manually.xml"
# Search parameters
PAGE = 146
TITLE = 'глюоний'
##################################################################



class Article:
    start_title = 0
    end_title = 0
    filename = ''


# Get filenames needed
filenames_raw = get_filenames(PAGES_DIR)
filenames = []
for i in range(PAGE, PAGE + 1):
    for filename in filenames_raw:
        beginning = "rp-" + str(i) + "_"
        if filename[:len(beginning)] == beginning and filename[-4:] == ".mmd":
            filenames.append(filename)


# Check for existing xml
filenames_raw = get_filenames(EXIT_DIR)
if not(EXIT_FILE in filenames_raw):
    root = ElementTree.Element('data')
    xml_write(root, EXIT_DIR + EXIT_FILE)


root = parse_xml(EXIT_DIR + EXIT_FILE)


# Add article title and metadata to xml tree
def add_article(article_local:Article, etree_root:ElementTree.Element, number:int):
    elem_article = ElementTree.SubElement(etree_root, 'article', {'n':str(number)})
    elem_title = ElementTree.SubElement(elem_article, 'title')
    elem_title.text = file[article_local.start_title + 1:article_local.end_title]
    elem_title_meta = ElementTree.SubElement(elem_article, 'title-meta')
    elem_title_file = ElementTree.SubElement(elem_title_meta, 'title-file')
    elem_title_file.text = article_local.filename
    elem_title_start = ElementTree.SubElement(elem_title_meta, 'title-start')
    elem_title_start.text = str(article_local.start_title + 1)
    elem_title_end = ElementTree.SubElement(elem_title_meta, 'title-end')
    elem_title_end.text = str(article_local.end_title)
    xml_write(etree_root, EXIT_DIR + EXIT_FILE)

# Read requested file
with codecs.open(PAGES_DIR + filenames[0], 'r', 'utf-8') as f:
    file = f.read()

# Find titles and add them
start_title = 0
end_title = 0
num = len(root) + 1
while file.find(TITLE, end_title) != -1:
    start_title = file.find(TITLE, start_title)
    end_title = start_title + len(TITLE)
    start_title -= 1 # Set on space before the title

    article = Article()
    article.start_title = max(start_title, 0)
    article.end_title = min(end_title, len(file))
    article.filename = filenames[0]
    add_article(article, root, num)

# 2. Исправление ошибок в заголовках

Состоит из двух частей: "составитель пар" и "подстановщик".

## 2.1. Составитель пар "оригинальный - исправленный" для заголовков

Формирует xml-список всех заголовков с возможными автоматическими исправлениями (в формате было / стало):
1. замена латиницы на аналогичную кириллицу;
2. замена заданных буквосочетаний (см. параметры)
3. удаление обрамляющих знаков препинания;
4. замена всех букв на заглавные (в том числе это избавляет дальнейшей необходимости исправлять имена);
5. слияние разорванных на отдельные буквы слов (если рядом оказываются несколько таких слов, то они оказываются слиты вместе).

Этот список необходимо просмотреть и исправить оставшиеся ошибки.

Дополнительно, для помощи в поиске орфографических ошибок, формируется строка с изменениями, предложенными спеллчекером. ВНИМАНИЕ: спеллчекер может делать ошибки в именах, специфических терминах и т.п., поэтому следует использовать его результаты лишь для ориентира.

In [ ]:
# 2.1. Составитель пар "оригинальный - исправленный" для заголовков:

############################ VARS ################################
WORK_DIR = "./matphys/"
INPUT_FILE = "FMEv2.xml"
CORRECTION_FILE = "FMEcorr.xml"
COMBINATIONS_CORR = dict_merge(COMBINATIONS_CORR_GLOBAL, {
    'ХК' : 'Ж',
    'ЬI' : 'Ы',
    'II' : 'Ш',
    'I' : 'П',
    'J' : 'Л',
    'ЛАГРАНХ' : 'ЛАГРАНЖ',
    'ЛАТРАНХ' : 'ЛАГРАНЖ',
})
SPELLCHECK_ONLY = True # Use if the only thing you need from this script is spellcheck
##################################################################


# Check for existing xml
filenames_raw = get_filenames(WORK_DIR)
if not(INPUT_FILE in filenames_raw):
    root = ElementTree.Element('data')
    xml_write(root, WORK_DIR + CORRECTION_FILE)


root = parse_xml(WORK_DIR + INPUT_FILE)


# Get all the titles into a dict
titles_dict = {}
pages_dict = {}
for article in root:
    title = get_xml_elem(article, 'title').text
    titles_dict[title] = (title, title)
    title_file = get_xml_elem(article, 'title-meta/title-file')
    pages_dict[title] = title_file.text[title_file.text.find('-')+1:title_file.text.find('_')]


if not SPELLCHECK_ONLY:
    # Correct preferred combinations and latin letters
    for title in titles_dict.keys():
        title_new = title_handle_latin(titles_dict[title][0], COMBINATIONS_CORR)
        titles_dict[title] = (title_new, title_new)

    # Remove bounding symbols
    for title in titles_dict.keys():
        title_new = title_handle_bounding(titles_dict[title][0])
        titles_dict[title] = (title_new, title_new)

    # CAPS
    for title in titles_dict.keys():
        title_new = titles_dict[title][0].upper()
        titles_dict[title] = (title_new, title_new)

    # Merge single-lettered words
    for title in titles_dict.keys():
        title_new = title_handle_merge(titles_dict[title][0])
        titles_dict[title] = (title_new, title_new)

    # Revert changes for aux formulas in titles
    for title in titles_dict.keys():
        title_new = title_handle_formulas(titles_dict[title][0], title)
        titles_dict[title] = (title_new, title_new)

# Try spellcheck on titles
spellcheck_dict_update()
for title in titles_dict.keys():
    title_new = titles_dict[title][0]
    title_suggestions = do_spellcheck(title_new)
    for i in range(len(title_new)):
        title_new = title_new[:i] + ('_' if title_new[i] not in [' ', '\n', '\r'] else title_new[i]) + (title_new[i+1:] if i + 1 <= len(title_new) else '')
    for pos in sorted(title_suggestions.keys(), reverse=True):
        title_new = title_new[:pos] + title_suggestions[pos][1] + title_new[pos+len(title_suggestions[pos][0]):]
    titles_dict[title] = (titles_dict[title][0], title_new)


# Write corrections xml
root = ElementTree.Element('data')
for i in titles_dict.items():
    pair = ElementTree.SubElement(root, 'pair')
    title_old = ElementTree.SubElement(pair, 'title_old')
    title_old.text = i[0]
    title_new = ElementTree.SubElement(pair, 'title_new')
    title_new.text = i[1][0]
    title_new = ElementTree.SubElement(pair, 'title__sc')
    title_new.text = i[1][1]
    page = ElementTree.SubElement(pair, 'page')
    page.text = pages_dict[i[0]]
xml_write(root, WORK_DIR + CORRECTION_FILE)

## 2.2. Подстановщик исправленных заголовков

Заменяет все заголовки на исправленные согласно списку пар.

In [ ]:
# 2.2. Подстановщик исправленных заголовков:

############################ VARS ################################
WORK_DIR = "./matphys/"
INPUT_FILE = "FMEv2.xml"
CORRECTION_FILE = "FMEcorr.xml"
EXIT_FILE = "FMEtitles.xml"
##################################################################



root = parse_xml(WORK_DIR + CORRECTION_FILE)


# Get all the corrections into a dict
titles_dict = {}
for pair in root:
    titles_dict[get_xml_elem(pair, 'title_old').text] = get_xml_elem(pair, 'title_new').text


root = parse_xml(WORK_DIR + INPUT_FILE)


# Replace titles
for article in root:
    get_xml_elem(article, 'title').text = titles_dict[get_xml_elem(article, 'title').text]
xml_write(root, WORK_DIR + EXIT_FILE)

# 3. Сортировщик / сливщик файлов с заголовками

Сортирует статьи в файлах из основного списка в порядке страница-расположение, т.е. (если не сказано иного) в алфавитном порядке и выводит в один выходной файл. Также порядковый номер заменяется uri формата "http://libmeta.ru/fme/article/1_Kraevaya". (Созданные uri кешируются по номеру страницы и позиции заголовка в тексте и при последующих запусках остаются неизменными, если включен `URI_SAFER`).

Также в конец выходного файл добавляются заголовки из "ручного" файла, в том же формате, но без сортировки, что позволяет добавлять случайно забытые статьи без изменения uri и имён файлов всех остальных статей.

In [48]:
# 3. Сортировщик / сливщик файлов с заголовками

############################ VARS ################################
WORK_DIR = "./results/"
TITLES_DIR = "FMEtitles/"
INPUT_FILES = ["FMEtitles-p5-100.xml", "FMEtitles-p101-200.xml", "FMEtitles-p201-300.xml", "FMEtitles-p301-400.xml", "FMEtitles-p301-400-add.xml",
               "FMEtitles-p401-500.xml", "FMEtitles-p501-600.xml", "FMEtitles-p601-692.xml", "FMEtitles-p601-692-add.xml"]
MANUALLY_ADDED_FILE = "FMEtitles-added-manually.xml"
URI_CACHE = "FMEtitles-uri-cache.xml"
EXIT_FILE = "FMEtitles-merged.xml"
# Uri safer prevents already existing uri from being changed. Set to False ONLY IF you need to update an existing uris.
URI_SAFER = True
##################################################################



class Article:
    title = ''
    start_title = ''
    end_title = ''
    filename = ''



# Try to get uri from the cache for title with given page and pos
def get_uri(title_page:str, title_pos:str) -> str:
    global cache_root
    for elem_uri in cache_root:
        if elem_uri.tag == 'uri' and elem_uri.attrib['page'] == title_page and elem_uri.attrib['pos'] == title_pos:
            return elem_uri.text
    return ''
# Cache given uri
def cache_uri(title_page:str, title_pos:str, uri_str:str):
    global cache_root
    elem_uri = ElementTree.SubElement(cache_root, 'uri', {'page':title_page, 'pos':title_pos})
    elem_uri.text = uri_str
    xml_write(cache_root, WORK_DIR + TITLES_DIR + URI_CACHE)


# Add article title and metadata to xml tree
def add_article(article_local:Article, etree_root:ElementTree.Element, number:int):
    page_str = article_local.filename[article_local.filename.find('-') + 1: article_local.filename.find('_')]
    uri_cached = get_uri(page_str, article_local.start_title)
    translitted = translit(article_local.title[:article_local.title.find(' ')], 'ru', True)
    while translitted.find('/') != -1:
        translitted = translitted[:translitted.find('/')] + '_' + translitted[translitted.find('/')+1:]		# Prevent slash being counted as subfolder in further
    uri_str = URI_PREFIX + "article/" + str(number) + "_" + translitted
    if URI_SAFER and uri_cached != '':
        uri_str = uri_cached
    else:
        cache_uri(page_str, article_local.start_title, uri_str)
    elem_article = ElementTree.SubElement(etree_root, 'article', {'uri':uri_str})
    elem_title = ElementTree.SubElement(elem_article, 'title')
    elem_title.text = article_local.title
    elem_title_meta = ElementTree.SubElement(elem_article, 'title-meta')
    elem_title_file = ElementTree.SubElement(elem_title_meta, 'title-file')
    elem_title_file.text = article_local.filename
    elem_title_start = ElementTree.SubElement(elem_title_meta, 'title-start')
    elem_title_start.text = str(int(article_local.start_title) + 1)
    elem_title_end = ElementTree.SubElement(elem_title_meta, 'title-end')
    elem_title_end.text = article_local.end_title


# Check for existing uri list
filenames_raw = get_filenames(WORK_DIR + TITLES_DIR)
if not(URI_CACHE in filenames_raw):
    root = ElementTree.Element('data')
    xml_write(root, WORK_DIR + TITLES_DIR + URI_CACHE)
cache_root = parse_xml(WORK_DIR + TITLES_DIR + URI_CACHE)


# Collect all the articles
print("Parsing main input files...")
articles_dict = {}
for filename in tqdm(INPUT_FILES):
    root = parse_xml(WORK_DIR + TITLES_DIR + filename)
    for article in root:
        title = get_xml_elem(article, 'title').text
        elem = get_xml_elem(article, 'title-meta/title-file')
        page = elem.text[elem.text.find('-')+1:elem.text.find('_')]
        pos = get_xml_elem(article, 'title-meta/title-start').text
        start = get_xml_elem(article, 'title-meta/title-start').text
        end = get_xml_elem(article, 'title-meta/title-end').text
        file = get_xml_elem(article, 'title-meta/title-file').text
        num = (int(page), int(pos))
        articles_dict[num] = {'title':title, 'file':file, 'start':start, 'end':end}

# Same for manually added articles
print("Parsing \"manual\" file...")
articles_dict_man = {}
nums_list_man = []
root = parse_xml(WORK_DIR + TITLES_DIR + MANUALLY_ADDED_FILE)
for article in root:
    title = get_xml_elem(article, 'title').text
    elem = get_xml_elem(article, 'title-meta/title-file')
    page = elem.text[elem.text.find('-')+1:elem.text.find('_')]
    pos = get_xml_elem(article, 'title-meta/title-start').text
    start = get_xml_elem(article, 'title-meta/title-start').text
    end = get_xml_elem(article, 'title-meta/title-end').text
    file = get_xml_elem(article, 'title-meta/title-file').text
    num = (int(page), int(pos))
    articles_dict_man[num] = {'title':title, 'file':file, 'start':start, 'end':end}
    nums_list_man.append(num)


# Sort keys and write articles accordingly
root = ElementTree.Element('data')
nums_list = sorted(list(i for i in articles_dict.keys()))
print("Writing main articles...")
for num in tqdm(range(len(nums_list))):
    article = Article()
    article.title = articles_dict[nums_list[num]]['title']
    article.start_title = articles_dict[nums_list[num]]['start']
    article.end_title = articles_dict[nums_list[num]]['end']
    article.filename = articles_dict[nums_list[num]]['file']
    add_article(article, root, num + 1)
for num in tqdm(range(len(nums_list_man))):
    article = Article()
    article.title = articles_dict_man[nums_list_man[num]]['title']
    article.start_title = articles_dict_man[nums_list_man[num]]['start']
    article.end_title = articles_dict_man[nums_list_man[num]]['end']
    article.filename = articles_dict_man[nums_list_man[num]]['file']
    add_article(article, root, num + 1 + len(nums_list))
xml_write(root, WORK_DIR + EXIT_FILE)

Parsing main input files...


  0%|          | 0/9 [00:00<?, ?it/s]

Parsing "manual" file...
Writing main articles...


  0%|          | 0/3584 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# 4. Парсер текстов статей

По информации из указанного файла с заголовками вытаскивает в сыром виде тексты статей. Каждая статья помещается в свой .xml файл, с именем, содержащим номер статьи и первое слово из заголовка транслитом.

In [49]:
# 4. Парсер текстов статей

############################ VARS ################################
TITLES_FILE = "./results/FMEtitles-merged.xml"
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./results/FMEarticles/"
COMBINATIONS_CORR = {
    'І' : 'I'		# This teo are different!
}
##################################################################


class Article:
    start_file = ''
    start_pos = 0
    end_file = ''
    end_pos = 0
    text = ''
    text_orig = ''
    uri = ''
    num = ''
    title = ''
    xml = ''

    def get_text(self):
        # Get filenames
        filenames_raw_local = get_filenames(PAGES_DIR)
        filenames_local = []
        for filename_local in filenames_raw_local:
            if filename_local[-4:] == ".mmd":
                filenames_local.append(filename_local)
        if self.start_file == self.end_file:
            with codecs.open(PAGES_DIR + self.start_file, 'r', 'utf-8') as f_in:
                self.text += f_in.read()[self.start_pos:self.end_pos]
        else:
            with codecs.open(PAGES_DIR + self.start_file, 'r', 'utf-8') as f_in:
                self.text += f_in.read()[self.start_pos:]
            for page_local in range(int(self.start_file[3:self.start_file.find('_')]) + 1, int(self.end_file[3:self.end_file.find('_')])):
                for filename_local in filenames_local:
                    if int(filename_local[3:filename_local.find('_')]) == page_local:
                        self.text += ' ' # Add a space to prevent word merging
                        with codecs.open(PAGES_DIR + filename_local, 'r', 'utf-8') as f_in:
                            self.text += f_in.read()
            self.text += ' ' # Add a space to prevent word merging
            with codecs.open(PAGES_DIR + self.end_file, 'r', 'utf-8') as f_in:
                self.text += f_in.read()[:self.end_pos]
        for comb_local in COMBINATIONS_CORR.keys():
            while self.text.find(comb_local) != -1:
                self.text = self.text[:self.text.find(comb_local)] + COMBINATIONS_CORR[comb_local] + self.text[self.text.find(comb_local) + len(comb_local):]
        while self.text is not None and len(self.text) and self.text[0] in [' ', ',', '.', ':', ';', '-', '\n', '\r']:
            self.text = self.text[1:]
        while self.text is not None and len(self.text) and self.text[-1] in [' ', '\n', '\r']:
            self.text = self.text[:-1]
        self.text_orig = self.text
        # Fix several capital symbols per word
        word_left = 0
        while word_left < len(self.text):
            word_right = min(len(self.text), self.text.find(' ', word_left) if self.text.find(' ', word_left) != -1 else len(self.text))
            word_right = min(word_right, self.text.find('\n', word_left) if self.text.find('\n', word_left) != -1 else len(self.text))
            word_right = min(word_right, self.text.find('\r', word_left) if self.text.find('\r', word_left) != -1 else len(self.text))
            word_right = min(word_right, self.text.find('-', word_left) if self.text.find('-', word_left) != -1 else len(self.text))
            word_right = min(word_right, self.text.find('.', word_left) if self.text.find('.', word_left) != -1 else len(self.text))
            word_str = self.text[word_left:word_right]
            if word_str is not None and len(word_str) > 1 and not check_in_uri(self.text, word_left) and not check_in_formula(self.text, word_left) and not check_in_link(self.text, word_left):
                word_str = word_str[0] + word_str[1:len(word_str)].lower()
                self.text = self.text[:word_left] + word_str + self.text[word_right:]
            word_left = word_right + 1

    def make_xml(self):
        self.get_text()

        elem_article = ElementTree.Element("article", {'uri':self.uri, 'alphabetic_pos':self.num})
        elem_title = ElementTree.SubElement(elem_article, 'title')
        elem_title.text = self.title
        elem_author = ElementTree.SubElement(elem_article, 'authors')
        elem_author.text = None
        elem_title_short = ElementTree.SubElement(elem_article, 'title_short')
        elem_title_short.text = None
        elem_pages = ElementTree.SubElement(elem_article, 'pages')
        elem_start = ElementTree.SubElement(elem_pages, 'start')
        elem_start.text = self.start_file[3:self.start_file.find('_', 3)]
        elem_end = ElementTree.SubElement(elem_pages, 'end')
        elem_end.text = self.end_file[3:self.end_file.find('_', 3)]
        elem_literature = ElementTree.SubElement(elem_article, 'literature')
        elem_literature_orig = ElementTree.SubElement(elem_literature, 'literature_orig')
        elem_literature_orig.text = None
        elem_formulas_remote = ElementTree.SubElement(elem_article, 'formulas_main')
        elem_formulas_remote.text = None
        elem_formulas_inline = ElementTree.SubElement(elem_article, 'formulas_aux')
        elem_formulas_inline.text = None
        elem_relations = ElementTree.SubElement(elem_article, 'relations')
        elem_relations.text = None
        elem_text = ElementTree.SubElement(elem_article, 'text')
        elem_text.text = self.text
        elem_text_orig = ElementTree.SubElement(elem_article, 'text_orig')
        elem_text_orig.text = self.text_orig

        self.xml = prettify(elem_article)



class Title:
    text = ''
    file = ''
    start_pos = 0
    end_pos = 0
    uri = ''


def get_titles_dict(etree_root:ElementTree.Element) -> dict:
    titles_dict_local = {}
    for elem_title in etree_root:
        elem_uri = elem_title.attrib['uri']
        elem_text = get_xml_elem(elem_title, 'title').text
        elem_file = get_xml_elem(elem_title, 'title-meta/title-file').text
        elem_page = int(elem_file[elem_file.find('-') + 1 : elem_file.find('_')])
        elem_start_pos = int(get_xml_elem(elem_title, 'title-meta/title-start').text)
        elem_end_pos = int(get_xml_elem(elem_title, 'title-meta/title-end').text)
        titles_dict_local[(elem_page, elem_start_pos)] = Title()
        titles_dict_local[(elem_page, elem_start_pos)].uri = elem_uri
        titles_dict_local[(elem_page, elem_start_pos)].text = elem_text
        titles_dict_local[(elem_page, elem_start_pos)].file = elem_file
        titles_dict_local[(elem_page, elem_start_pos)].start_pos = elem_start_pos
        titles_dict_local[(elem_page, elem_start_pos)].end_pos = elem_end_pos
    return titles_dict_local


def get_title(number:int, dict_with_titles:dict) -> Title:
    out_title = Title()
    titles_dict_keys = sorted(dict_with_titles.keys())
    for p in range(len(titles_dict_keys)):
        if p == number:
            out_title = dict_with_titles[titles_dict_keys[p]]
    return out_title


root = parse_xml(TITLES_FILE)

# Create articles list
articles_list = []
title = Title()
titles_dict = get_titles_dict(root)
print("Getting articles info...")
for i in tqdm(range(len(root))):
    title = get_title(i, titles_dict)
    if i:
        articles_list[-1].end_file = title.file
        articles_list[-1].end_pos = max(title.start_pos - 2, 0) # There is a shift for some reason
    articles_list.append(Article())
    articles_list[-1].uri = title.uri
    articles_list[-1].num = str(i + 1)
    articles_list[-1].title = title.text
    articles_list[-1].start_file = title.file
    articles_list[-1].start_pos = title.end_pos
    articles_list[-1].end_file = title.file
    with codecs.open(PAGES_DIR + title.file, 'r', 'utf-8') as f:
        articles_list[-1].end_pos = len(f.read())

# Parse texts themselves and write
print("Parsing articles...")
for i in tqdm(range(len(articles_list))):
    articles_list[i].make_xml()
    with codecs.open(EXIT_DIR + '' + articles_list[i].uri[len(URI_PREFIX) + 8:] + '.xml', 'w', 'utf-8') as f:
        f.write(articles_list[i].xml)

Getting articles info...


  0%|          | 0/3586 [00:00<?, ?it/s]

Parsing articles...


  0%|          | 0/3586 [00:00<?, ?it/s]

# 5. Проверка правописания в текстах

## 5.1. Сканер

Сканирует тексты из указанного диапазона статей и выносит все показавшиеся подозрительными слова в отдельный xml следующего формата:
- Статья (имя файла в аттрибутах)
  - Слово (позиция в тексте и флаги в аттрибутах)
    - Исходный вариант
    - Контекстная строка (размер задаётся в разделе параметров скрипта)
    - Предложенная замена

Предлагается два флага для определения дальнейшей "судьбы" слова: "результат" (0 -- исходное, 1 -- предложенное) и "добавление в словарь" (0 -- не добавлять, 1 -- добавить как есть, 2 -- перевести в нижний регистр и добавить (для первого слова в предложении), 3 -- сделать первую букву заглавной и добавить (для имён, случайно распознанных без заглавной); применяется к выбранному результату)

In [50]:
# 5.1. Проверка правописания в текстах. Сканер.

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
EXIT_DIR = "./matphys/"
CONTEXT_SIZE = 20
START_ARTICLE = 701
END_ARTICLE = 750
# Flags for usual cases
DEFAULT_RESULT_FLAG = '1'
DEFAULT_ADD_TO_PWL_FLAG = '0'
# Flags is name is detected
"""NAME_RESULT_FLAG = '0'
NAME_ADD_TO_PWL_FLAG = '1'"""
# Cases that have to be overriden
OVERRIDE_FORCE_CYRILLIC = {
    'Ссср' : 'СССР',
    'Церн' : 'ЦЕРН'
}
OVERRIDE_AS_IS = {
}
##################################################################


spellcheck_dict_update()

# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)
#filenames = ['4_ABELEVA.xml']

root = ElementTree.Element('data')

total_wois = 0
for filename in tqdm(filenames):
    article_number = int(filename[:filename.find('_')])
    if article_number < START_ARTICLE or article_number > END_ARTICLE:
        continue

    #print(f'{filename}: found ', end='')
    article = parse_xml(ARTICLES_DIR + filename)
    text = get_xml_elem(article, 'text')

    #add_to_pwl(filename[filename.find('_')+1:filename.find('.xml')])

    text_suggestions = do_spellcheck(text.text)
    #print(len(text_suggestions.keys()))
    total_wois += len(text_suggestions.keys())
    if len(text_suggestions.keys()):
        article = ElementTree.SubElement(root, 'article', {'filename': filename})
        for pos in text_suggestions.keys():
            #print(f'{pos}: {text_suggestions[pos][0]} -> {text_suggestions[pos][1]}')
            local_result_flag = DEFAULT_RESULT_FLAG
            local_add_to_pwl_flag = DEFAULT_ADD_TO_PWL_FLAG
            # Process possible name case
            '''if len(text_suggestions[pos][0]) >= 2 and len(text_suggestions[pos][1]) >= 2:
                is_name_orig = re.match(r"[А-ЯA-Z]", text_suggestions[pos][0][0]) is not None and re.match(r"[а-яa-z]", text_suggestions[pos][0][1]) is not None
                is_name_sugg = re.match(r"[А-ЯA-Z]", text_suggestions[pos][1][0]) is not None and re.match(r"[а-яa-z]", text_suggestions[pos][1][1]) is not None
                if is_name_orig and is_name_sugg:
                    local_result_flag = NAME_RESULT_FLAG
                    local_add_to_pwl_flag = NAME_ADD_TO_PWL_FLAG'''
            # Override specific cases
            suggestion_text = text_suggestions[pos][1]
            if title_handle_latin(text_suggestions[pos][0], COMBINATIONS_CORR_GLOBAL) in OVERRIDE_FORCE_CYRILLIC.keys():
                suggestion_text = OVERRIDE_FORCE_CYRILLIC[title_handle_latin(text_suggestions[pos][0], COMBINATIONS_CORR_GLOBAL)]
                local_result_flag = DEFAULT_RESULT_FLAG
                local_add_to_pwl_flag = DEFAULT_ADD_TO_PWL_FLAG
            if text_suggestions[pos][0] in OVERRIDE_AS_IS.keys():
                suggestion_text = OVERRIDE_AS_IS[text_suggestions[pos][0]]
                local_result_flag = DEFAULT_RESULT_FLAG
                local_add_to_pwl_flag = DEFAULT_ADD_TO_PWL_FLAG
            word = ElementTree.SubElement(article, 'word', {'pos': str(pos), 'result': local_result_flag, 'add_to_pwl': local_add_to_pwl_flag})
            source = ElementTree.SubElement(word, 'source')
            source.text = text_suggestions[pos][0]
            context = ElementTree.SubElement(word, 'context')
            context_string = text.text[max(0, pos - CONTEXT_SIZE):min(len(text.text), pos + len(text_suggestions[pos][0]) + CONTEXT_SIZE)]
            while context_string.find('\n') != -1:
                context_string = context_string[:context_string.find('\n')] + '\\n' + context_string[context_string.find('\n')+1:]
            while context_string.find('\r') != -1:
                context_string = context_string[:context_string.find('\r')] + '\\r' + context_string[context_string.find('\r')+1:]
            context.text = context_string
            suggestion = ElementTree.SubElement(word, 'suggestion')
            suggestion.text = suggestion_text

print("Total cases found:", total_wois)


with codecs.open(EXIT_DIR + f'FMEspellcheck-a{START_ARTICLE}-{END_ARTICLE}.xml', 'w', 'utf-8') as f:
    f.write(prettify(root))

  0%|          | 0/3586 [00:00<?, ?it/s]

Total cases found: 115


## 5.2. Пополнение словаря

Добавляет отмеченные флагом "добавление в словарь" слова из всех файлов в директории спеллчека
- Учитывается, было ли выбрано оригинальное слово или исправленное флагом "результат".
- Словарь сортируется по алфавиту при каждом запуске.
- Дубликаты удаляются при каждом запуске (символы разного регистра одинаковыми не считаются).
- Слова добавленные вручную при запуске не удаляются.

Чтобы объединить ваш словарь с другим, скопируйте и вставьте всё содержимое нового словаря в ваш, после чего запустите скрипт. Дубликаты будут удалены, итоговый словарь будет отсортирован.

In [51]:
# 5.2. Проверка правописания в текстах. Пополнение словаря.

############################ VARS ################################
SPELLCHECK_DIR = "./results/FMEspellcheck/"
##################################################################


# Read PWL and form word list
with codecs.open(PERSONAL_WORD_LIST, 'r', 'utf-8') as f:
    PWL_text = f.read()
additions = [i.strip() for i in PWL_text.split('\n')]
while '' in additions:
    additions.remove('')
PWL_text = ''

# Read all spellcheck outputs and create additions list
# Get filenames needed
filenames = get_filenames(SPELLCHECK_DIR)

print("Scanning for PWL additions...")
for filename in tqdm(filenames):
    root = parse_xml(SPELLCHECK_DIR + filename)
    for article in root:
        if article.tag == "article":
            for word in article:
                if word.tag == "word" and word.attrib["add_to_pwl"] != '0':
                    word_text = get_xml_elem(word, 'suggestion').text.strip() if word.attrib["result"] == '1' else get_xml_elem(word, 'source').text.strip()
                    if word.attrib["add_to_pwl"] == '2':
                        additions.append(word_text.lower())
                    elif word.attrib["add_to_pwl"] == '3' and len(word_text):
                        additions.append(word_text[0].upper() + word_text[1:] if len(word_text) > 1 else '')
                    else:
                        additions.append(word_text)

# Make new PWL list and sort it
print("Processing PWL...")
PWL_list_new = []
for word in tqdm(additions):
    # Check (and correct) that the word has no latin and cyrillic letters at the same time
    if word is not None and len(word):
        exist_from_comb = False
        exist_rus = False
        for i in range(len(word)):
            exist_from_comb = True if word[i] in COMBINATIONS_CORR_GLOBAL.keys() else exist_from_comb
            exist_rus = True if re.match(r"[А-Яа-я]", word[i]) is not None else exist_rus
        if exist_from_comb and exist_rus:
            for i in range(len(word)):
                word = word[:i] + (COMBINATIONS_CORR_GLOBAL[word[i]] if word[i] in COMBINATIONS_CORR_GLOBAL.keys() else word[i]) + (word[i+1:] if (i + 1) <= len(word) else '')
    # Append word to the list if not present yet
    if word is not None and len(word) and not word in PWL_list_new:
        PWL_list_new.append(word)
PWL_list_new.sort()

# Write PWL
print("Writing PWL...")
for word in tqdm(PWL_list_new):
    PWL_text = PWL_text + word + '\n'
with codecs.open(PERSONAL_WORD_LIST, 'w', 'utf-8') as f:
    f.write(PWL_text)

Scanning for PWL additions...


  0%|          | 0/11 [00:00<?, ?it/s]

Processing PWL...


  0%|          | 0/2782 [00:00<?, ?it/s]

Writing PWL...


  0%|          | 0/1878 [00:00<?, ?it/s]

## 5.3. Подстановка исправленной орфографии

Подставляет в исходный текст исправленные слова или оригиналы, в зависимости от установленного флага "результат".

In [52]:
# 5.3. Проверка правописания в текстах. Подстановка исправленной орфографии.

############################ VARS ################################
SPELLCHECK_DIR = "./results/FMEspellcheck/"
ARTICLES_DIR = "./results/FMEarticles/"
##################################################################


# Get filenames needed
filenames = get_filenames(SPELLCHECK_DIR)

for filename_data in tqdm(filenames):
    # Parse articles corrections file
    root_data = parse_xml(SPELLCHECK_DIR + filename_data)
    for article in root_data:
        if article.tag == 'article':
            filename_article = article.attrib['filename']
            root_article = parse_xml(ARTICLES_DIR + filename_article)
            text = get_xml_elem(root_article, 'text')
            # Parse corrections in one article
            words = []
            for word in article:
                if word.tag == 'word':
                    pos = int(word.attrib['pos'])
                    len_src = len(get_xml_elem(word, 'source').text)
                    word_text = get_xml_elem(word, 'suggestion').text if word.attrib['result'] == '1' else get_xml_elem(word, 'source').text
                    words.append((pos, word_text, len_src))
            words.sort(reverse=True)
            # Apply corrections
            for word in words:
                pos = word[0]
                len_src = word[2]
                word_text = word[1]
                text.text = text.text[:pos] + word_text + (text.text[pos+len_src:] if pos+len_src <= len(text.text) else '')
            # Write corrected article xml
            with codecs.open(ARTICLES_DIR + filename_article, 'w', 'utf-8') as f:
                f.write(prettify(root_article))

  0%|          | 0/11 [00:00<?, ?it/s]

# 6. Парсер авторов статьи

Ищет в конце текста статей конструкции типа ` [Xxxx]. [Xxxx]. [Xxxx]` или ` [Xxxx].[Xxxx]. [Xxxx]` и интерпретирует её как автора статьи.

In [53]:
# 6. Парсинг авторов статьи

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
COMBINATIONS_CORR = dict_merge(COMBINATIONS_CORR_UNICODE, {
    'II' : 'П'
})
LOCAL_DICT = {'0':'О', '3':'З', '6':'б'}
##################################################################


# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)

for filename in tqdm(filenames):
    article = parse_xml(ARTICLES_DIR + filename)
    textelem = get_xml_elem(article, 'text')
    text = textelem.text
    authors = get_xml_elem(article, 'authors')

    auth_start = 1
    auth_list = []
    while auth_start and text is not None:
        # Find first non-space from the end
        while text[-1] == ' ' or text[-1] == '\n' or text[-1] == '\r':
            text = text[:-1]

        auth_start = 0
        # Try recognize
        first_space = max(text.rfind(' ', 0, len(text)), text.rfind('\n', 0, len(text)), text.rfind('\r', 0, len(text)))
        second_space = max(text.rfind(' ', 0, first_space), text.rfind('\n', 0, first_space), text.rfind('\r', 0, first_space))
        third_space = max(text.rfind(' ', 0, second_space), text.rfind('\n', 0, second_space), text.rfind('\r', 0, second_space))
        if first_space >= 0 and text[first_space-1] == '.' and second_space >= 0:
            if text.find('.', second_space, first_space-1) != -1: # If there's no space between initials
                third_space = second_space
                second_space = first_space
            if text[second_space-1] == '.' and third_space >= 0:
                # Check if first letters of each word are capitals
                keep = text
                for comb in LOCAL_DICT.keys():
                    while text[third_space+1:].find(comb) != -1:
                        text = text[:third_space+1+text[third_space+1:].find(comb)] + LOCAL_DICT[comb] + text[third_space+2+text[third_space+1:].find(comb):]
                if re.match(r"[A-ZА-ЯІ]", text[first_space+1]) is not None and re.match(r"[A-ZА-ЯІ]", text[second_space+1]) is not None and re.match(r"[A-ZА-ЯІ]", text[third_space+1]) is not None:
                    auth_start = third_space + 1
                text = keep

        if auth_start: # Suggest that an article cannot consist of author only and therefore auth_start should be > 0
            #print(article.attrib['uri'], author_text)
            author_text = text[auth_start:]
            if author_text[author_text.find('.')+1] != ' ': # Add space if there's no one between initials
                author_text = author_text[:author_text.find('.')+1] + ' ' + author_text[author_text.find('.')+1:]
            if author_text[-1] == '.' or author_text[-1] == ',':
                author_text = author_text[:-1]
            # convert wrong symbols
            for comb in dict_merge(COMBINATIONS_CORR, LOCAL_DICT).keys():
                while author_text.find(comb) != -1:
                    author_text = author_text[:author_text.find(comb)] + dict_merge(COMBINATIONS_CORR, LOCAL_DICT)[comb] + author_text[author_text.find(comb) + len(comb):]

            auth_list.append(author_text)
            text = text[:auth_start]

    # add authors, reverse their order to alphabetic
    for auth in reversed(auth_list):
        author = ElementTree.SubElement(authors, 'author')
        author.text = auth

    textelem.text = text
    with codecs.open(ARTICLES_DIR + filename, 'w', 'utf-8') as f:
        f.write(prettify(article))

  0%|          | 0/3586 [00:00<?, ?it/s]

# 7. Парсер литературы

После извлечения авторов статьи в конце за текстом статьи присутствует только строчка литературы, если вообще присутствует. Поэтому ищется и извлекается фрагмент начиная с "`Лит.:`". Он разделяется на сегменты по "`[num]`", а сегменты на подфрагменты по запятым. Общий вид сегмента полагается следующим: "`[Авторы (возможно несколько, определяются по наличию инициалов в конце подфрагмента)], Название (возможно содержит запятые), Номер тома (может отсутствовать), [Информация об издании (может частично или полностью отсутствовать)], Год, [Прочее (главы, страницы и прочее, может отсутствовать)];`"

In [54]:
# 7. Парсинг литературы

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
COMBINATIONS_CORR_LOCAL = dict_merge(dict_merge(COMBINATIONS_CORR_ALPHABET, COMBINATIONS_CORR_UNICODE), {'J':'Л'})
##################################################################


class Unit:
    authors = []
    title = ""
    publication = ""
    year = ""
    other = ""


# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)

for filename in tqdm(filenames):
    article = parse_xml(ARTICLES_DIR + filename)
    textelem = get_xml_elem(article, 'text')
    text = textelem.text
    literature = get_xml_elem(article, 'literature')
    literature_orig = get_xml_elem(literature, 'literature_orig')

    if textelem.text is not None and len(textelem.text):
        #Find literature start position and extract if present
        for key in COMBINATIONS_CORR_LOCAL.keys():
            while text.find(key) != -1:
                text = text[:text.find(key)] + COMBINATIONS_CORR_LOCAL[key] + text[text.find(key)+1:]
        text = text.upper()
        lit_pos = text.rfind('\nЛИТ.: ')
        lit_pos = text.rfind('\rЛИТ.: ') if lit_pos == -1 else lit_pos
        lit_pos = text.rfind(' ЛИТ.: ') if lit_pos == -1 else lit_pos
        if lit_pos != -1:
            literature_orig.text = textelem.text[lit_pos:]
            while literature_orig.text[0] in [' ', '\n', '\r']:
                literature_orig.text = literature_orig.text[1:]
            textelem.text = textelem.text[:lit_pos]
            while textelem.text[-1] in [' ', '\n', '\r']:
                textelem.text = textelem.text[:-1]


            # Parse literature string
            text = literature_orig.text
            units = []
            num = 1
            while text.find('['+str(num)+']') != -1:
                units.append(text[text.find('['+str(num)+']')+len('['+str(num)+']'):(text.find('['+str(num+1)+']') if text.find('['+str(num+1)+']') != -1 else len(text))])
                num += 1
            for unit in units:
                logical_parts = Unit()
                logical_parts.authors.clear()
                subunits = unit.split(',')
                while '' in subunits:
                    subunits.remove('')
                pos_last_auth = -1
                pos_last_title = -1
                pos_thome = -1
                pos_transl = -1
                pos_pub_num = -1
                pos_pub_place = -1
                pos_year = -1


                # Define positions of most common pats of literature string
                for i in range(len(subunits)):
                    text = subunits[i]
                    while text[-1] in [' ', '\n', '\r', ';']:
                        text = text[:-1]
                    while text[0] in [' ', '\n', '\r']:
                        text = text[1:]
                    subunits[i] = text

                    if pos_last_auth + 1 == i: # Recognize authors
                        keep = text
                        pos_initials = 0
                        for j in range(len(text)):
                            if text[j] in COMBINATIONS_CORR_UNICODE:
                                text = text[:j] + COMBINATIONS_CORR_UNICODE[text[j]] + text[j+1:]
                        if text[-1] == '.' and re.match(r"[[А-ЯA-Z]", text[-2]) is not None and text[-3] == ' ' and text[-4] == '.' and re.match(r"[[А-ЯA-Z]", text[-5]) is not None:
                            # "X. X."
                            pos_last_auth = i
                            pos_initials = -5
                        elif text[-1] == '.' and re.match(r"[[А-ЯA-Z]", text[-2]) is not None and text[-3] == '.' and re.match(r"[[А-ЯA-Z]", text[-4]) is not None:
                            # "X.X."
                            pos_last_auth = i
                            text = text[:-2] + ' ' + text[-2:]
                            pos_initials = -5
                        elif text[-1] == '.' and re.match(r"[[А-ЯA-Z]", text[-2]) is not None:
                            # "X."
                            pos_last_auth = i
                            pos_initials = -2
                        else: # Title starts
                            text = keep
                        # If correct
                        if pos_last_auth == i:
                            surname = text[:pos_initials]
                            while surname.find(' ') != -1:
                                surname = surname[:surname.find(' ')] + surname[surname.find(' ')+1:]
                            text = surname + ' ' + text[pos_initials:]
                            j = 1
                            while j < len(text):
                                if re.match(r"[А-ЯA-Z]", text[j]) is not None and re.match(r"[а-яa-z]", text[j-1]) is not None:
                                    text = text[:j] + ' ' + text[j:]
                                    j = 1
                                else:
                                    j += 1
                            subunits[i] = text
                    else:
                        if pos_thome == -1: # Recognize thome
                            keep = text
                            for j in range(len(text)):
                                if text[j] in COMBINATIONS_CORR_GLOBAL:
                                    text = text[:j] + COMBINATIONS_CORR_GLOBAL[text[j]] + text[j+1:]
                            if text.upper().find('Т.') != -1:
                                pos_last_title = (i - 1) if pos_last_title == -1 else pos_last_title
                                pos_thome = i
                            text = keep
                        if pos_transl == -1: # Recognize publication number
                            keep = text
                            for j in range(len(text)):
                                if text[j] in COMBINATIONS_CORR_GLOBAL:
                                    text = text[:j] + COMBINATIONS_CORR_GLOBAL[text[j]] + text[j+1:]
                            if text.upper().find('ПЕР.') != -1:
                                pos_last_title = (i - 1) if pos_last_title == -1 else pos_last_title
                                pos_transl = i
                            text = keep
                        if pos_pub_num == -1: # Recognize publication number
                            keep = text
                            for j in range(len(text)):
                                if text[j] in COMBINATIONS_CORR_GLOBAL:
                                    text = text[:j] + COMBINATIONS_CORR_GLOBAL[text[j]] + text[j+1:]
                            if text.upper().find('ИЗД.') != -1:
                                pos_last_title = (i - 1) if pos_last_title == -1 else pos_last_title
                                pos_pub_num = i
                            text = keep
                        if pos_pub_place == -1: # Recognize publication place
                            keep = text
                            for j in range(len(text)):
                                if text[j] in COMBINATIONS_CORR_GLOBAL:
                                    text = text[:j] + COMBINATIONS_CORR_GLOBAL[text[j]] + text[j+1:]
                            if text.upper() in ['М.', 'Л.', 'СПБ.', 'М.Л.', 'Л.М.', 'М.СПБ.', 'СПБ.М.']:
                                pos_last_title = (i - 1) if pos_last_title == -1 else pos_last_title
                                pos_pub_place = i
                            text = keep
                        # If correct
                        if pos_last_auth != i and (pos_thome == i or pos_pub_num == i or pos_pub_place == i):
                            for j in range(len(text)):
                                if text[j] in COMBINATIONS_CORR_UNICODE:
                                    subunits[i] = text[:j] + COMBINATIONS_CORR_UNICODE[text[j]] + text[j+1:]

                        if pos_year == -1 and len(text) >= 4: # Recognize year
                            numbers = ['0','1','2','3','4','5','6','7','8','9']
                            j = 0
                            for j in range(len(text) - 3):
                                if text[j] in numbers and text[j+1] in numbers and text[j+2] in numbers and text[j+3] in numbers:
                                    pos_last_title = (i - 1) if pos_last_title == -1 else pos_last_title
                                    pos_year = i
                                    break
                            # if correct
                            if pos_year == i:
                                subunits[i] = text[j:j+4]


                # Extract info from literature string using positions defined above
                for i in range(len(subunits)):
                    text = subunits[i]
                    if pos_last_auth >= i: # Author
                        logical_parts.authors.append(text)
                    elif pos_last_auth < i <= pos_last_title: # Title
                        logical_parts.title = logical_parts.title + ('' if len(logical_parts.title) == 0 else ', ') + text
                    elif pos_year == i: # Year
                        logical_parts.year = logical_parts.year + ('' if len(logical_parts.year) == 0 else ', ') + text
                    elif ((pos_pub_num <= i and pos_pub_num != -1) or (pos_pub_place <= i and pos_pub_place != -1) or (pos_transl <= i and pos_transl != -1) or (pos_thome + 1 <= i and pos_thome != -1)) and pos_year > i: # Publication
                        logical_parts.publication = logical_parts.publication + ('' if len(logical_parts.publication) == 0 else ', ') + text
                    else: # Other
                        logical_parts.other = logical_parts.other + ('' if len(logical_parts.other) == 0 else ', ') + text


                # Debug section
                """print('\n', filename, unit)
                print('authors:', logical_parts.authors)
                print('title:', logical_parts.title)
                print('publication:', logical_parts.publication)
                print('year:', logical_parts.year)
                print('other:', logical_parts.other)
                print(pos_last_auth, pos_last_title, pos_thome, pos_transl, pos_pub_num, pos_pub_place, pos_year)"""


                # Add literature unit
                unit = ElementTree.SubElement(literature, "unit")
                for auth_str in logical_parts.authors:
                    author = ElementTree.SubElement(unit, "author")
                    author.text = auth_str
                title = ElementTree.SubElement(unit, "title")
                title.text = logical_parts.title
                publication = ElementTree.SubElement(unit, "publication")
                publication.text = logical_parts.publication
                year = ElementTree.SubElement(unit, "year")
                year.text = logical_parts.year
                other = ElementTree.SubElement(unit, "other")
                other.text = logical_parts.other


            # Write xml
            with codecs.open(ARTICLES_DIR + filename, 'w', 'utf-8') as f:
                f.write(prettify(article))

  0%|          | 0/3586 [00:00<?, ?it/s]

# 8. Парсер формул

Выносит из текстов ранее подготовленных xml-файлов статей сначала выносные, а затем строчные формулы, оставляя на их месте ссылку внутри математического окружения.

Минимальная длина в символах, которой должна обладать строчная формула, настраивается.

In [55]:
# 8. Парсер формул

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
MIN_INLINE_LEN = 0
##################################################################


# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)

n_main = 0
n_aux = 0
for filename in tqdm(filenames):
    article = parse_xml(ARTICLES_DIR + filename)
    #print('REMOTES: ' + article.attrib['uri'])
    text = get_xml_elem(article, 'text')
    formulas_main = get_xml_elem(article, 'formulas_main')
    formulas_aux = get_xml_elem(article, 'formulas_aux')

    # Get main formulas
    pos_find = 0
    pos_start = 0
    pos_end = 0
    n = 1
    while text.text is not None and text.text.find('\\[', pos_find) != -1:
        pos_start = text.text.find('\\[', pos_find) + 2
        pos_end = text.text.find('\\]', pos_start)
        while text.text[pos_start] == '\n':
            pos_start += 1
        while text.text[pos_end-1] == '\n':
            pos_end -= 1
        pos_find = pos_start
        uri = URI_PREFIX + 'formula/main' + article.attrib['uri'][article.attrib['uri'].rfind('/', 0, article.attrib['uri'].find('_')):article.attrib['uri'].find('_')+1] + str(n) + article.attrib['uri'][article.attrib['uri'].find('_'):]
        n += 1
        formula = ElementTree.SubElement(formulas_main, 'formula', {'uri':uri})
        formula.text = text.text[pos_start:pos_end]
        text.text = text.text[:pos_start] + 'URI[[' + uri + ']]/URI' + text.text[pos_end:]
    n_main += n

    # Get auxiliary formulas
    pos_find = 0
    pos_start = 0
    pos_end = 0
    cnt = 0
    n = 1
    # Count dollar symbols
    while text.text is not None and text.text.find('$', pos_find) != -1:
        pos_find = text.text.find('$', pos_find) + 1
        cnt += 1
    # If cnt is not even assume that first one is garbage from title
    pos_find = 0
    if cnt % 2:
        pos_find = text.text.find('$', pos_find)
        text.text = text.text[:pos_find] + '#' + text.text[pos_find+1:]
    while text.text is not None and text.text.find('$', pos_find) != -1:
        pos_start = text.text.find('$', pos_find) + 1
        pos_end = text.text.find('$', pos_start)
        if not check_in_uri(text.text, pos_start) and not check_in_uri(text.text, pos_end):
            while text.text[pos_start] == '\n':
                pos_start += 1
            while text.text[pos_end-1] == '\n':
                pos_end -= 1
            pos_find = pos_start
            if pos_end - pos_start >= MIN_INLINE_LEN:
                uri = URI_PREFIX + 'formula/aux' + article.attrib['uri'][article.attrib['uri'].rfind('/', 0, article.attrib['uri'].find('_')):article.attrib['uri'].find('_')+1] + str(n) + article.attrib['uri'][article.attrib['uri'].find('_'):]
                n += 1
                formula = ElementTree.SubElement(formulas_aux, 'formula', {'uri':uri})
                formula.text = text.text[pos_start:pos_end]
                text.text = text.text[:pos_start] + 'URI[[' + uri + ']]/URI' + text.text[pos_end:]
            pos_find = text.text.find('$', pos_find) + 1
        else:
            pos_find = pos_end + 1
    n_aux += n

    with codecs.open(ARTICLES_DIR + filename, 'w', 'utf-8') as f:
        f.write(prettify(article))

print("Found main formulas:", n_main)
print("Found auxiliary formulas:", n_aux)

  0%|          | 0/3586 [00:00<?, ?it/s]

Found main formulas: 9959
Found auxiliary formulas: 38421


## 8.1. Вынос формул

Выносит все формулы в отдельный файл с указанием типа для возможной последующей обработки.

In [56]:
# 8.1. Вынос формул

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
EXIT_FILE = "./results/FMEformulas.xml"
##################################################################


# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)


formulas = ElementTree.Element('formulas')

for filename in tqdm(filenames):
    root = parse_xml(ARTICLES_DIR + filename)
    fmain = get_xml_elem(root, 'formulas_main')
    faux = get_xml_elem(root, 'formulas_aux')

    for formula in fmain:
        formulas.append(formula)
    for formula in faux:
        formulas.append(formula)

with codecs.open(EXIT_FILE, 'w', 'utf-8') as f:
    f.write(prettify(formulas))

  0%|          | 0/3586 [00:00<?, ?it/s]

## 8.2. Проверка формул

Случайным образом выбирает 20 случайных формул (из случайных статей) и вставляет их в математическое окружение Markdown для визуальной проверки

In [57]:
# 8.2. Проверка формул

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
EXIT_FILE = "./matphys/FMEformulas_check.md"
NUMBER = 20
##################################################################


# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)


file = ''

i = 0
while i < NUMBER:
    root = parse_xml(ARTICLES_DIR + filenames[randint(0, len(filenames)-1)])

    # Get all the info from article
    fmain = get_xml_elem(root, 'formulas_main')
    start = get_xml_elem(root, 'pages/start').text


    # if there's no formulas in the article try another one
    total_num = 0
    for formula in fmain:
        total_num += 1
    if not total_num:
        continue
    i += 1

    num = randint(0, 100) % total_num

    formula = fmain[num].text

    file += f'{i}. Статья: {root.attrib["uri"]}, Начало на стр. {start}, формула {num + 1}:\n$${formula}$$\n'

with codecs.open(EXIT_FILE, 'w', 'utf-8') as f:
    f.write(file)

# 9. Парсер ссылок типа "смотри также"

Ищет в тексте ссылки начинающиеся на `"см. [другие опциональные вводные слова]"` и пытается найти соответствующие им статьи в энциклопедии.

In [58]:
# 9. Парсер ссылок типа "смотри также"

# -------------------------- VARS --------------------------------
ARTICLES_DIR = "./results/FMEarticles/"
STRICT_SEQUENCING = False  # DEFAULT: False; Links with changed order of words will also be found if False
BRUTE_FORCE_MODE = False  # Maximum amount of links to find, but takes more time (very slow, use with multiprocessing)
USE_MULTIPROCESSING = False  # WARNING: Does not work inside Jupyter!!!; Significantly speeds up scanning process
KEEP_FREE = 0  # Make multiprocessing keep free a specified amount of logic processors, if needed.
# ----------------------------------------------------------------


from lib import *

number_of_processors = max(1, os.cpu_count() - KEEP_FREE)
matches_list = []


# Find previous word beginning - 1 from given position
def find_prev_space(work_text: str, start_pos: int) -> int:
    new_pos = 0
    for sym in ' \n\r':
        new_pos_s = work_text.rfind(sym, 0, start_pos)
        new_pos = max(new_pos, (0 if new_pos_s == -1 else new_pos_s))
    return new_pos


# Find next word ending + 1 from given position
def find_next_space(work_text: str, start_pos: int) -> int:
    new_pos = len(work_text)
    for sym in ' \n\r':
        new_pos_s = work_text.find(sym, start_pos + 1)
        new_pos = min(new_pos, (len(work_text) if new_pos_s == -1 else new_pos_s))
    return new_pos


# Try to find a matching title to the given one
def find_matching_title(sequence_list: list) -> (bool, bool, bool, int):
    match_possible_local = False
    global matches_list
    global titles_list
    # Check with titles from matches list
    for list_pos in range(len(titles_list)):
        if list_pos in matches_list:
            match_local = False
            title_sel = titles_list[list_pos]
            # No match if the title is shorter than the sequence
            if len(title_sel) >= len(sequence_list):
                title_positions = [p for p in range(len(title_sel))]
                len_diff = len(title_positions) - len(sequence_list)
                # Check each word in given sequence
                for p in range(len(sequence_list)):
                    match_current = False
                    # Check if any word form title matches the current word from the sequence
                    for position in title_positions:
                        seq_item = sequence_list[p]
                        if STRICT_SEQUENCING:
                            position = title_positions[0]  # Force straight title words sequencing
                        title_item = title_sel[position]
                        # Try to negotiate word endings
                        seq_len = len(seq_item)
                        title_len = len(title_item)
                        max_len = max(len(title_item), len(seq_item))
                        if seq_len > 5 and title_len > 5 and abs(seq_len - title_len) <= 1:
                            cut_len = min(max_len - 3, 4)
                            seq_item = seq_item[:cut_len]
                            title_item = title_item[:cut_len]
                        # Found matching word
                        if seq_item == title_item:
                            match_current = True
                            title_positions.remove(position)  # Each word from title can be met only once
                            break
                    # If no matching word found then there is no local match
                    if not match_current:
                        break
                match_local = len(title_positions) == len_diff  # Match found for every word in sequence
                match_possible_local = match_possible_local or match_local
            # Remove the title position from matches list if no match for sure
            if not match_local:
                matches_list.remove(list_pos)
            # Exact match for the given sequence
            if match_local and len(title_sel) == len(sequence_list):
                return True, True, True, list_pos

    # If only one local match consider possible solid match where title is longer than the link sequence
    if len(matches_list) == 1:
        return True, True, False, matches_list[0]

    # No solid match found
    return match_possible_local, False, False, -1


# Function for multiprocessing speedup
def loop(filenames_loc: list) -> int:
    global matches_list
    n_loc = 0
    for filename_loc in tqdm(filenames_loc):
        article_loc = parse_xml(ARTICLES_DIR + filename_loc)
        textelem_loc = get_xml_elem(article_loc, 'text')
        text_loc = textelem_loc.text
        n_loc_loc = 0

        if text_loc is not None and len(text_loc):
            # Move along the text from right to left to allow easier uri insertion
            find_right_loc = len(text_loc)
            find_left_loc = find_prev_space(text_loc, find_right_loc)
            _border_left_loc = find_right_loc + 1

            # Find link starting word
            while find_left_loc != -1:
                if check_in_uri(text_loc, find_left_loc):
                    find_right_loc = find_left_loc
                    find_left_loc = find_prev_space(text_loc, find_left_loc) if find_left_loc else -1
                    continue
                word_loc = title_handle_latin(text_loc[find_left_loc:find_right_loc].strip(' \n\r.,;:!?\\()[]{}&'),
                                              COMBINATIONS_CORR_GLOBAL).upper()
                # 'm' can be interpreted both as 'M' and 'T'
                if word_loc == 'СМ' or word_loc == 'СТ' or BRUTE_FORCE_MODE:
                    border_left_loc = find_left_loc if BRUTE_FORCE_MODE else find_right_loc
                    while border_left_loc < len(text_loc) and text_loc[border_left_loc] in ' \n\r.,;:!?\\()[]{}&':
                        border_left_loc += 1
                    if _border_left_loc <= border_left_loc:
                        # Same position as before
                        break
                    border_right_f_loc = border_left_loc
                    _border_right_loc = border_left_loc
                    border_find_allowed_loc = True
                    _match_single_loc = False
                    _match_pos_loc = -1
                    matches_list = [p for p in range(len(titles_list))]
                    _event_parts_len = 0
                    while border_find_allowed_loc:
                        border_right_f_loc = find_next_space(text_loc, border_right_f_loc)
                        border_right_loc = border_right_f_loc
                        while text_loc[border_right_loc - 1] in ' \n\r.,;:!?\\()[]{}&':
                            border_right_loc -= 1
                        if border_right_loc - border_left_loc < 2:
                            _border_left_loc = border_left_loc
                            break
                        border_find_allowed_loc = False if border_right_f_loc == len(text_loc) else True
                        event_loc = title_handle_latin(
                            text_loc[border_left_loc:border_right_loc], COMBINATIONS_CORR_GLOBAL).upper()
                        if event_loc in ['В', 'ПРИ', 'ТАКЖЕ', 'В СТ', 'ПРИ СТ', 'SЕЕАLSО', 'SАMЕАS'] and \
                                not BRUTE_FORCE_MODE:
                            # Starting words continuation
                            continue

                        # Extract word sequence and try to find a matching title from list
                        event_parts_loc = event_loc.split(' ')
                        # Check for punctuation in event
                        if len(event_parts_loc) and len(event_parts_loc[-1]):
                            part_loc = event_parts_loc[-1]
                            for sym in '\n\r\\':
                                if sym in part_loc:
                                    # Severe punctuation
                                    _border_left_loc = border_left_loc
                                    break
                        if len(event_parts_loc) > 1:
                            part_loc = event_parts_loc[-1]
                            if len(part_loc) and (part_loc[0] in '\n\r([{'):
                                # Severe punctuation
                                _border_left_loc = border_left_loc
                                break
                            part_loc = event_parts_loc[-2]
                            if len(part_loc) and (part_loc[0] in '\n\r([{' or part_loc[-1] in ',;:!?\\)]}') \
                                    or (len(part_loc) > 2 and part_loc[-1] == '.'):
                                # Severe punctuation
                                _border_left_loc = border_left_loc
                                break
                        for part_loc in event_parts_loc:
                            if part_loc == '' or part_loc in ' \n\r.,;:!?\\()[]{}&':
                                event_parts_loc.remove(part_loc)
                        if _event_parts_len == len(event_parts_loc):
                            # Nothing new here
                            continue
                        _event_parts_len = len(event_parts_loc)
                        # Check for junk sequences
                        not_junk_loc = 0
                        _not_junk_loc = 0
                        for part_loc in event_parts_loc:
                            _not_junk_loc = not_junk_loc
                            not_junk_loc += 1 if len(part_loc.strip(' \n\r.,;:!?\\()[]{}&')) > 5 else 0
                        if not_junk_loc < 2 and len(event_parts_loc) >= 2:
                            not_junk_loc = 0
                        if _not_junk_loc < 2 and len(event_parts_loc) >= 2:
                            _not_junk_loc = 0

                        (match_possible_loc, match_single_loc, match_exact_loc, match_pos_loc) = find_matching_title(
                            [p.strip(' \n\r.,;:!?\\()[]{}&') for p in event_parts_loc])
                        border_find_allowed_loc = border_find_allowed_loc and match_possible_loc
                        # Remember if single match
                        if match_single_loc:
                            _border_right_loc = border_right_loc
                            _match_pos_loc = match_pos_loc
                        # Consider last single match as exact
                        if (not match_single_loc and _match_single_loc and _not_junk_loc) or (
                                not border_find_allowed_loc and match_single_loc and not_junk_loc):
                            border_right_loc = _border_right_loc
                            match_pos_loc = _match_pos_loc
                            match_exact_loc = True

                        _match_single_loc = match_single_loc
                        # Process exact match
                        if match_exact_loc:
                            _match_single_loc = False
                            # Add an inter-link
                            n_loc_loc += 1
                            uri_pos_us = article_loc.attrib['uri'].find('_')
                            uri_pos_sl = article_loc.attrib['uri'].rfind('/', 0, uri_pos_us)
                            uri_loc = URI_PREFIX + 'relation' + \
                                      article_loc.attrib['uri'][uri_pos_sl:uri_pos_us + 1] + \
                                      str(n_loc_loc) + \
                                      article_loc.attrib['uri'][uri_pos_us:]
                            relations_loc = get_xml_elem(article_loc, 'relations')
                            relation_loc = ElementTree.SubElement(relations_loc, 'relation', {'uri': uri_loc})
                            rel_text_loc = ElementTree.SubElement(relation_loc, 'rel_text')
                            rel_text_loc.text = text_loc[border_left_loc:border_right_loc]
                            rel_tgt_loc = ElementTree.SubElement(relation_loc, 'target')
                            related_article_loc = parse_xml(ARTICLES_DIR + articles_list[match_pos_loc])
                            rel_tgt_loc.text = related_article_loc.attrib['uri']
                            text_loc = text_loc[:border_left_loc] + 'URI[[' + uri_loc + ']]/URI' + text_loc[
                                                                                                   border_right_loc:]
                            # Continue in case of multilink
                            if not BRUTE_FORCE_MODE:
                                border_left_loc += len('URI[[' + uri_loc + ']]/URI')
                                while border_left_loc < len(text_loc) and not text_loc[border_left_loc] in ' \n\r':
                                    border_left_loc += 1
                                border_right_f_loc = border_left_loc
                                matches_list = [p for p in range(len(titles_list))]
                            else:
                                _border_left_loc = border_left_loc
                                break
                        elif not border_find_allowed_loc:
                            _border_left_loc = border_right_loc

                find_right_loc = find_left_loc
                find_left_loc = find_prev_space(text_loc, find_left_loc) if find_left_loc else -1

        # Write xml
        textelem_loc.text = text_loc
        xml_write(article_loc, ARTICLES_DIR + filename_loc)
        n_loc += n_loc_loc

    return n_loc


# Get filenames needed
filenames = get_filenames(ARTICLES_DIR)

# Get all the titles into a list
articles_list = []
titles_list = []
if __name__ != '__main__' or not USE_MULTIPROCESSING:

    # DEBUG                           ###
    # filenames = ["1583_LI.xml", "2471_PUASSONA.xml", "2472_PUASSONA.xml"]
    #####################################
    if __name__ == '__main__':
        print("Preparing search base...")
    for filename in tqdm(filenames):
        article = parse_xml(ARTICLES_DIR + filename)
        title = get_xml_elem(article, 'title').text
        titles_list.append([title_word.upper().strip(' \n\r.,;:!?\\()[]{}&') for title_word in title.split(' ')])
        articles_list.append(filename)

if __name__ == '__main__':

    # DEBUG                           ###
    # filenames = ['3576_JaKOBI.xml']    ###
    #####################################
    print("Searching relations in articles...")
    if USE_MULTIPROCESSING:
        arrays = []
        with Pool(number_of_processors) as pool:
            for i in range(number_of_processors):
                arr_len = len(filenames)
                arr_start = i * (arr_len // number_of_processors)
                arr_end = ((i + 1) * (arr_len // number_of_processors)) if i + 1 < number_of_processors else arr_len
                arrays.append(filenames[arr_start:arr_end])
            n = sum(pool.map(loop, arrays))
    else:
        n = loop(filenames)
    print("Relations found in total:", n)
    if USE_MULTIPROCESSING:
        input()

Preparing search base...


  0%|          | 0/3586 [00:00<?, ?it/s]

Searching relations in articles...


  0%|          | 0/3586 [00:00<?, ?it/s]

Relations found in total: 1969
